In [1]:
# !pip install xlrd
# !pip install spacy

# https://github.com/Alex-CHUN-YU/Word2vec
# !pip install gensim
# !pip install jieba
# !pip install thulac

# !pip install textblob
# !pip install snownlp

In [2]:
import numpy as np
import pandas as pd
import xlrd
import thulac

In [3]:
data = "data/dataset.xlsx"
xls_data = pd.read_excel(data)

In [4]:
xls_data.head()

,content,label
0,【曼市恐襲】耶耶托尼捐50萬助受害者 經理人：曼市無分曼城曼聯 面對恐襲，這一刻足球不再重要...,C
1,《人民日報》：「特事特辦」還是少些好 特首梁振英幼女梁頌昕遺失行李，由航空公司職員到禁區外代...,A
2,特朗普、佩林雙劍合璧 變身政壇「雙頭怪」？ 特朗普和佩林看似風馬牛不相及，但其實兩人在政治立...,B
3,【球場上的世界公民】講座精華節錄 2017年3月25日，港學堂與法國文化協會共同舉辦《世界公...,C
4,【再見忠臣】迪拿達利告別烏甸尼斯 懷緬最動人12年 烏甸尼斯Vs卡比 5月16日（周一） 2...,C


In [26]:
# xls_data['content'][0]
categories = set(xls_data['label'])

with open('categories.pickle', 'wb') as handle:
    pickle.dump(categories, handle)

In [6]:
len(xls_data)

3894

In [7]:
%%time
from bs4 import BeautifulSoup
import thulac

thu = thulac.thulac(seg_only=True)

def clean_and_cut(text):
    soup = BeautifulSoup(text, 'html.parser')
    new_text = soup.get_text()
    
    new_text_cut = thu.cut(new_text, text=True).replace("\n", "")
    return new_text_cut

xls_data['content_splited'] = xls_data['content'].apply(clean_and_cut)

Model loaded succeed
CPU times: user 1min 31s, sys: 490 ms, total: 1min 31s
Wall time: 1min 32s


In [8]:
# thu = thulac.thulac(seg_only=True)
# def ch_text_cut(t):
#     return thu.cut(t)

# xls_data['content_splited'] = xls_data['content'].apply(ch_text_cut)
# xls_data['content_splited'][4].split(' ')

In [9]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(xls_data['content_splited'], xls_data['label'], 
#                                                     test_size=0.25, random_state=42)
# full_train = [(k, v) for k, v in zip(X_train, y_train)]
# full_test = [(k, v) for k, v in zip(X_test, y_test)]

# for k, v in zip(X_train[0:10], y_train[0:10]):
#     print("Key:" + k)
#     print("\n")
#     print("Value:" + v)
#     print("\n")

# full_train[0:5]

In [10]:
# %%time
# from textblob.classifiers import NaiveBayesClassifier
# import nltk
# nltk.download("punkt")
# cl = NaiveBayesClassifier(full_train)
# cl.show_informative_features(20)

In [11]:
# %%time
# print("Accuracy: ", cl.accuracy(full_test))

In [12]:
import pickle
from keras.preprocessing.text import Tokenizer

num_words = 3
tokenizer = Tokenizer(num_words=num_words)

Using TensorFlow backend.


In [13]:
# tokenizer.fit_on_texts(xls_data['content_splited'])
# tokenizer.texts_to_matrix(xls_data['content_splited'])

In [14]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.metrics import top_k_categorical_accuracy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import keras

In [15]:
%%time
num_classes = len(set(xls_data['label']))

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(xls_data['label'])
# print(integer_encoded)

# Category encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)

CPU times: user 4.9 ms, sys: 1.94 ms, total: 6.84 ms
Wall time: 5 ms


In [16]:
%%time
tokenizer.fit_on_texts(xls_data['content_splited'])
# X = tokenizer.texts_to_sequences(xls_data['content_splited'])

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

X = tokenizer.texts_to_matrix(xls_data['content_splited'])
vocab_size = len(tokenizer.word_index) + 1

CPU times: user 3.29 s, sys: 48.5 ms, total: 3.34 s
Wall time: 3.34 s


In [17]:
%%time
from keras.preprocessing.sequence import pad_sequences

max_words = np.max([len(x.split(" ")) for x in xls_data['content_splited']])
X = pad_sequences(X, maxlen=max_words, padding='post')

# saving
with open('max_words.pickle', 'wb') as handle:
    pickle.dump(max_words, handle, protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 263 ms, sys: 104 ms, total: 367 ms
Wall time: 364 ms


In [18]:
%%time
# print(X.shape)
# print(y.shape)

# from keras.wrappers.scikit_learn import KerasClassifier

def top_1_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=1)

def baseline_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 32, input_length= X.shape[1]))

    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.2))

    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.2))

    model.add(Flatten())
    
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', top_1_accuracy])

    model.summary()
    
    return model

# estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=100, verbose=1)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.06 µs


In [19]:
%%time

# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

model = baseline_model()

from keras.callbacks import ModelCheckpoint 
checkPoint = ModelCheckpoint(filepath='text_model.hdf5', verbose=1, save_best_only=True)

model.fit(X, y, batch_size=128, epochs=10, validation_split=0.2, shuffle=True, callbacks=[checkPoint])
model.load_weights('text_model.hdf5')

model.save("text_full_model.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7647, 32)          4671808   
_________________________________________________________________
dense_1 (Dense)              (None, 7647, 64)          2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 7647, 64)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 7647, 128)         8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 7647, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 978816)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 2936451   
Total para

In [20]:
%%time
model.predict_proba(X)

3894/3894 [==============================] - 19s    
CPU times: user 1min 22s, sys: 5.39 s, total: 1min 27s
Wall time: 19 s


array([[ 0.23976718,  0.22282586,  0.53740704],
       [ 0.23976718,  0.22282586,  0.53740704],
       [ 0.23976718,  0.22282586,  0.53740704],
       ..., 
       [ 0.23976718,  0.22282586,  0.53740704],
       [ 0.23976718,  0.22282583,  0.53740704],
       [ 0.23976718,  0.22282583,  0.53740704]], dtype=float32)

In [21]:
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

with open('max_words.pickle', 'rb') as handle:
    load_max_words = pickle.load(handle)
    
example_text = """
再發霜凍警告21人低溫入院兩人危殆

寒冷天氣持續，天文台在下午再度發出霜凍警告，指明早新界北部可能出現地面霜。寒冷天氣警告及紅色火災危險警告繼續生效。天文台預測，明早市區最低氣溫約8度，新界再低兩三度，日間最高氣溫約13度。有菜農指，若果結霜，只能提早收割。

醫管局指，在截至下午五時的過去二十四小時，公立醫院合共收到二十一名病人因低溫而需送往急症室治療，包括九男十二女，年齡介乎五十三至九十三歲，當中兩人危殆、六人嚴重。
"""
new_text = clean_and_cut(example_text)
print(new_text)

new_X = load_tokenizer.texts_to_matrix(new_text)
new_X = pad_sequences(new_X, maxlen=load_max_words, padding='post')
print(new_X)

再 發 霜 凍 警告 21 人 低 溫 入院 兩 人 危殆寒冷 天氣 持續 ， 天文台 在 下午 再度 發出霜 凍 警告 ， 指明 早 新界 北部 可能 出現 地面霜 。 寒冷 天氣 警告 及 紅色 火災 危險 警告 繼續 生效 。 天文台 預測 ， 明早市 區 最低 氣溫約 8 度 ， 新界 再 低 兩 三 度 ， 日間 最高 氣溫約 13 度 。 有 菜 農指 ， 若 果結霜 ， 只能 提 早 收割 。醫管局 指 ， 在 截至 下午 五時 的 過 去 二十四 小 時 ， 公立 醫院 合共 收到 二十一 名 病人 因 低溫 而 需 送 往 急症室 治療 ， 包括 九 男 十二 女 ， 年齡 介乎 五十三 至 九十三 歲 ， 當 中 兩 人 危殆 、 六 人 嚴重 。
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [22]:
%%time
predict = model.predict(new_X)
predict_prob = model.predict_proba(new_X)
predict_class = model.predict_classes(new_X)

338/338 [==============================] - 1s     
CPU times: user 21 s, sys: 1.38 s, total: 22.4 s
Wall time: 4.94 s


In [23]:
# print(predict[np.argmax(predict)])
# print(predict_prob[np.argmax(predict_prob)])
# print(predict_class[np.argmax(predict_class)])
from collections import Counter
cls_counter = Counter(predict_class)

In [24]:
idx = cls_counter.most_common(1)[0][0]

In [25]:
np.sort(list(categories))[2]

'C'